In [1]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy
import string
import random
from sklearn import linear_model

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import torch

In [4]:
def assertFloat(x):
    assert type(float(x)) == float

def assertFloatList(items, N):
    assert len(items) == N
    assert [type(float(x)) for x in items] == [float]*N

In [5]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [6]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [7]:
answers = {}

In [8]:
# Some data structures that will be useful

In [9]:
allRatings = []
for l in readCSV("train_Interactions.csv.gz"):
    allRatings.append(l)

In [10]:
len(allRatings)

200000

In [11]:
ratingsTrain = allRatings[:190000]
ratingsValid = allRatings[190000:]
ratingsPerUser = defaultdict(list)
ratingsPerItem = defaultdict(list)
bookUsers = defaultdict(set)
userBooks = defaultdict(set)
for u,b,r in ratingsTrain:
    ratingsPerUser[u].append((b,r))
    ratingsPerItem[b].append((u,r))
    bookUsers[b].add(u)
    userBooks[u].add(b)

In [12]:
##################################################
# Read prediction                                #
##################################################

In [13]:
# Copied from baseline code
bookCount = defaultdict(int)
totalRead = 0

for user,book,_ in readCSV("train_Interactions.csv.gz"):
    bookCount[book] += 1
    totalRead += 1

mostPopular = [(bookCount[x], x) for x in bookCount]
mostPopular.sort()
mostPopular.reverse()

return1 = set()
count = 0
for ic, i in mostPopular:
    count += ic
    return1.add(i)
    if count > totalRead/2: break

In [14]:
### Question 1

In [15]:
negative_samples = []
for user, book, _ in ratingsValid:
    while True:
        negative_book = random.choice(list(bookCount.keys()))
        if all(b != negative_book for b, _ in ratingsPerUser[user]):
            negative_samples.append((user, negative_book, 0))  # 0 for negative sample
            break

validation_with_negatives = [(u, b, 1) for u, b, _ in ratingsValid] + negative_samples

correct_predictions = 0
for user, book, label in validation_with_negatives:
    prediction = 1 if book in return1 else 0
    if prediction == label:
        correct_predictions += 1

acc1 = correct_predictions / len(validation_with_negatives)
print(f"model accuracy on valid: {acc1}")

model accuracy on valid: 0.71125


In [16]:
answers['Q1'] = acc1

In [17]:
assertFloat(answers['Q1'])

In [18]:
### Question 2

In [19]:
threshold = None
acc2 = 0

for fraction in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    curr_threshold = totalRead * fraction

    popular_books = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        popular_books.add(i)
        if count > curr_threshold:
            break

    correct_predictions = 0
    for user, book, label in validation_with_negatives:
        prediction = 1 if book in popular_books else 0
        if prediction == label:
            correct_predictions += 1

    curr_acc = correct_predictions / len(validation_with_negatives)
    
    if curr_acc > acc2:
        acc2 = curr_acc
        threshold = fraction

print(f"Best threshold: {threshold}")
print(f"Best accuracy: {acc2}")

Best threshold: 0.7
Best accuracy: 0.7526


In [20]:
answers['Q2'] = [threshold, acc2]

In [21]:
assertFloat(answers['Q2'][0])
assertFloat(answers['Q2'][1])

In [22]:
### Question 3/4

In [23]:
def jaccard_similarity(book1, book2):
    users1 = bookUsers.get(book1, set())
    users2 = bookUsers.get(book2, set())
    if not users1 or not users2:
        return 0
    intersection = len(users1.intersection(users2))
    union = len(users1.union(users2))
    return intersection / union

best_threshold = None
best_accuracy = 0

for threshold in range(20, 40, 2):
    threshold = threshold / 10000
    correct_predictions = 0

    for user, book, label in validation_with_negatives:
        # find max jaccard sim between books
        max_jaccard_sim = max([jaccard_similarity(book, b_read) for b_read in userBooks[user]], default=0)
        
        # predict as read if above threshold
        prediction = 1 if max_jaccard_sim > threshold else 0

        if prediction == label:
            correct_predictions += 1

    accuracy = correct_predictions / len(validation_with_negatives)

    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_threshold = threshold

    print(threshold, accuracy)


print(f"Best Jaccard similarity threshold: {best_threshold}")
print(f"Accuracy with best Jaccard threshold: {best_accuracy}")



0.002 0.6986
0.0022 0.70045
0.0024 0.70315
0.0026 0.70365
0.0028 0.70365
0.003 0.70355
0.0032 0.70205
0.0034 0.70135
0.0036 0.70215
0.0038 0.7019
Best Jaccard similarity threshold: 0.0026
Accuracy with best Jaccard threshold: 0.70365


In [24]:
acc3 = best_accuracy

In [25]:
# Popularity-based threshold tuning
pop_threshold = None
pop_accuracy = 0

for fraction in [0.68, 0.7, 0.72, 0.74]:
    curr_pop_threshold = totalRead * fraction

    # Select popular books based on threshold
    popular_books = set()
    count = 0
    for ic, i in mostPopular:
        count += ic
        popular_books.add(i)
        if count > curr_pop_threshold:
            break

    # Jaccard-based threshold tuning within popular book criteria
    for jaccard_threshold in range(0, 10):
        jaccard_threshold = jaccard_threshold / 100
        correct_predictions = 0

        for user, book, label in validation_with_negatives:
            # Check popularity criterion first
            if book in popular_books:
                prediction = 1
            else:
                max_jaccard_sim = max([jaccard_similarity(book, b_read) for b_read in userBooks[user]], default=0)
                prediction = 1 if max_jaccard_sim > jaccard_threshold else 0
            
            if prediction == label:
                correct_predictions += 1

        accuracy = correct_predictions / len(validation_with_negatives)

        # Track best combination of popularity and Jaccard thresholds
        if accuracy > pop_accuracy:
            pop_accuracy = accuracy
            pop_threshold = (fraction, jaccard_threshold)

        print(f"Popularity fraction: {fraction}, Jaccard threshold: {jaccard_threshold}, Accuracy: {accuracy}")

print(f"Best popularity fraction: {pop_threshold[0]}")
print(f"Best Jaccard similarity threshold: {pop_threshold[1]}")
print(f"Combined accuracy with best thresholds: {pop_accuracy}")


Popularity fraction: 0.68, Jaccard threshold: 0.0, Accuracy: 0.69775
Popularity fraction: 0.68, Jaccard threshold: 0.01, Accuracy: 0.74925
Popularity fraction: 0.68, Jaccard threshold: 0.02, Accuracy: 0.7515
Popularity fraction: 0.68, Jaccard threshold: 0.03, Accuracy: 0.75205
Popularity fraction: 0.68, Jaccard threshold: 0.04, Accuracy: 0.75165
Popularity fraction: 0.68, Jaccard threshold: 0.05, Accuracy: 0.752
Popularity fraction: 0.68, Jaccard threshold: 0.06, Accuracy: 0.7518
Popularity fraction: 0.68, Jaccard threshold: 0.07, Accuracy: 0.75185
Popularity fraction: 0.68, Jaccard threshold: 0.08, Accuracy: 0.75175
Popularity fraction: 0.68, Jaccard threshold: 0.09, Accuracy: 0.7518
Popularity fraction: 0.7, Jaccard threshold: 0.0, Accuracy: 0.69735
Popularity fraction: 0.7, Jaccard threshold: 0.01, Accuracy: 0.74895
Popularity fraction: 0.7, Jaccard threshold: 0.02, Accuracy: 0.75245
Popularity fraction: 0.7, Jaccard threshold: 0.03, Accuracy: 0.75295
Popularity fraction: 0.7, Jacca

In [26]:
acc4 = pop_accuracy

In [27]:
answers['Q3'] = acc3
answers['Q4'] = acc4

In [28]:
acc3, acc4

(0.70365, 0.75375)

In [29]:
assertFloat(answers['Q3'])
assertFloat(answers['Q4'])

In [30]:
predictions = open("predictions_Read.csv", 'w')

curr_pop_threshold = totalRead * 0.72
jaccard_threshold = 0.05

# Select popular books based on threshold
popular_books = set()
count = 0
for ic, i in mostPopular:
    count += ic
    popular_books.add(i)
    if count > curr_pop_threshold:
        break

for l in open("pairs_Read.csv"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,b = l.strip().split(',')

    if b in popular_books:
        prediction = 1
    else:
        max_jaccard_sim = max([jaccard_similarity(b, b_read) for b_read in userBooks[u]], default=0)
        prediction = 1 if max_jaccard_sim > jaccard_threshold else 0
    
    predictions.write(u + ',' + b + ',' + str(prediction) + '\n')

predictions.close()

In [31]:
answers['Q5'] = "I confirm that I have uploaded an assignment submission to gradescope"

In [32]:
assert type(answers['Q5']) == str

In [33]:
##################################################
# Rating prediction                              #
##################################################

In [34]:
### Question 6

In [35]:
import numpy as np

In [36]:
class BiasOnlyModel:
    def __init__(self, lamb, convergence_threshold=1e-4):
        self.alpha = 0.0
        self.betaU = defaultdict(float)
        self.betaI = defaultdict(float)
        self.lamb = lamb
        self.convergence_threshold = convergence_threshold

    def fit(self, train_data, num_epochs=50):
        ratings = [d[2] for d in train_data]
        self.alpha = np.mean(ratings)

        previous_mse = float('inf')

        for epoch in range(num_epochs):
            alpha_grad = 0.0
            betaU_grad = defaultdict(float)
            betaI_grad = defaultdict(float)

            for user, item, rating in train_data:
                pred = self.predict(user, item)
                error = rating - pred

                alpha_grad += -2 * error
                betaU_grad[user] += -2 * error
                betaI_grad[item] += -2 * error

            for user in betaU_grad:
                betaU_grad[user] += 2 * self.lamb * self.betaU[user]
            for item in betaI_grad:
                betaI_grad[item] += 2 * self.lamb * self.betaI[item]

            self.alpha -= alpha_grad / len(train_data)
            for user in betaU_grad:
                self.betaU[user] -= betaU_grad[user] / len(train_data)
            for item in betaI_grad:
                self.betaI[item] -= betaI_grad[item] / len(train_data)

            mse_train = self.calculate_mse(train_data)
            print(f"Epoch {epoch + 1}, Training MSE: {mse_train}")

            if abs(previous_mse - mse_train) < self.convergence_threshold:
                print("Convergence achieved.")
                break

            previous_mse = mse_train

    def predict(self, user, item):
        return self.alpha + self.betaU[user] + self.betaI[item]

    def calculate_sse(self, data):
        sse = 0.0
        for user, item, rating in data:
            pred = self.predict(user, item)
            sse += (rating - pred) ** 2

        sse += self.lamb * (sum(b ** 2 for b in self.betaU.values()) + sum(b ** 2 for b in self.betaI.values()))
        return sse

    def calculate_mse(self, data):
        sse = self.calculate_sse(data)
        return sse / len(data)


In [94]:
model = BiasOnlyModel(lamb=1)

model.fit(ratingsTrain, num_epochs=200)

validMSE = model.calculate_mse(ratingsValid)
print("Validation MSE:", validMSE)

Epoch 1, Training MSE: 1.7385038717138568
Epoch 2, Training MSE: 1.7379082555901695
Epoch 3, Training MSE: 1.7373174540127907
Epoch 4, Training MSE: 1.7367314026164713
Epoch 5, Training MSE: 1.7361500380262862
Epoch 6, Training MSE: 1.7355732978415486
Epoch 7, Training MSE: 1.7350011206193425
Epoch 8, Training MSE: 1.734433445858994
Epoch 9, Training MSE: 1.733870213986438
Epoch 10, Training MSE: 1.7333113663391335
Epoch 11, Training MSE: 1.7327568451505382
Epoch 12, Training MSE: 1.7322065935360416
Epoch 13, Training MSE: 1.7316605554780022
Epoch 14, Training MSE: 1.7311186758117287
Epoch 15, Training MSE: 1.7305809002112948
Epoch 16, Training MSE: 1.7300471751759166
Epoch 17, Training MSE: 1.7295174480164321
Epoch 18, Training MSE: 1.7289916668420284
Epoch 19, Training MSE: 1.7284697805469877


KeyboardInterrupt: 

In [37]:
# Collect unique user and item IDs from both training and validation data
user_ids = set()
item_ids = set()
for u, b, r in ratingsTrain + ratingsValid:
    user_ids.add(u)
    item_ids.add(b)

# Create mappings from IDs to indices
user2index = {user_id: idx for idx, user_id in enumerate(sorted(user_ids))}
item2index = {item_id: idx for idx, item_id in enumerate(sorted(item_ids))}
num_users = len(user2index)
num_items = len(item2index)

In [38]:
def prepare_data(data, user2index, item2index):
    user_indices = torch.tensor([user2index[u] for u, _, _ in data], dtype=torch.long)
    item_indices = torch.tensor([item2index[b] for _, b, _ in data], dtype=torch.long)
    ratings = torch.tensor([float(r) for _, _, r in data], dtype=torch.float)
    return user_indices, item_indices, ratings

train_users, train_items, train_ratings = prepare_data(ratingsTrain, user2index, item2index)
val_users, val_items, val_ratings = prepare_data(ratingsValid, user2index, item2index)

In [39]:
alpha = torch.nn.Parameter(torch.tensor(0.0))
beta_user = torch.nn.Parameter(torch.zeros(num_users))
beta_item = torch.nn.Parameter(torch.zeros(num_items))

In [44]:
optimizer = torch.optim.LBFGS([alpha, beta_user, beta_item])
lambda_reg = 1.0  # Regularization parameter

def closure():
    optimizer.zero_grad()
    predictions = alpha + beta_user[train_users] + beta_item[train_items]
    errors = train_ratings - predictions
    loss = (errors ** 2).sum()
    reg_term = lambda_reg * (beta_user ** 2).sum() + lambda_reg * (beta_item ** 2).sum()
    total_loss = loss + reg_term
    total_loss.backward()
    return total_loss

In [49]:
optimizer.step(closure)

with torch.no_grad():
    val_predictions = alpha + beta_user[val_users] + beta_item[val_items]
    val_errors = val_ratings - val_predictions
    val_mse = (val_errors ** 2).mean()
    print("Validation MSE:", val_mse.item())


Validation MSE: 1.4422765970230103


In [50]:
answers['Q6'] = float(val_mse)

In [51]:
answers['Q6']

1.4422765970230103

In [52]:
assertFloat(answers['Q6'])

In [53]:
# Access the beta_user tensor
beta_user_values = beta_user.detach()

# Find the indices of the largest and smallest beta_u
idx_max = torch.argmax(beta_user_values).item()
idx_min = torch.argmin(beta_user_values).item()

# Create an inverse mapping from index to user ID
index2user = {idx: user_id for user_id, idx in user2index.items()}

# Get the user IDs
user_id_max = index2user[idx_max]
user_id_min = index2user[idx_min]

# Get the beta_u values
beta_max = beta_user_values[idx_max].item()
beta_min = beta_user_values[idx_min].item()

# Report the results
print(f"User with largest beta_u: User ID = {user_id_max}, beta_u = {beta_max}")
print(f"User with smallest beta_u: User ID = {user_id_min}, beta_u = {beta_min}")

User with largest beta_u: User ID = u79275096, beta_u = 1.7030433416366577
User with smallest beta_u: User ID = u88024921, beta_u = -3.7170488834381104


In [54]:
### Question 7
maxUser = user_id_max
maxBeta = beta_max
minUser = user_id_min
minBeta = beta_min

In [55]:
answers['Q7'] = [maxUser, minUser, maxBeta, minBeta]

In [56]:
answers['Q7']

['u79275096', 'u88024921', 1.7030433416366577, -3.7170488834381104]

In [57]:
assert [type(x) for x in answers['Q7']] == [str, str, float, float]

In [58]:
### Question 8

In [61]:
lambda_values = [0.001, 0.01, 0.1, 1, 5, 10, 20, 50, 100]
results = []

for lambda_reg in lambda_values:
    print(f"\nTesting lambda = {lambda_reg}")
    # Initialize parameters
    alpha = torch.nn.Parameter(torch.tensor(0.0))
    beta_user = torch.nn.Parameter(torch.zeros(num_users))
    beta_item = torch.nn.Parameter(torch.zeros(num_items))
    
    # Set up optimizer
    optimizer = torch.optim.LBFGS([alpha, beta_user, beta_item])
    
    # Define objective function
    def closure():
        optimizer.zero_grad()
        predictions = alpha + beta_user[train_users] + beta_item[train_items]
        errors = train_ratings - predictions
        loss = (errors ** 2).sum()
        # Regularization term (excluding alpha)
        reg_term = lambda_reg * ((beta_user ** 2).sum() + (beta_item ** 2).sum())
        total_loss = loss + reg_term
        total_loss.backward()
        return total_loss
    
    # Train the model
    optimizer.step(closure)
    
    # Evaluate on validation set
    with torch.no_grad():
        val_predictions = alpha + beta_user[val_users] + beta_item[val_items]
        val_errors = val_ratings - val_predictions
        val_mse = (val_errors ** 2).mean().item()
        print(f"Validation MSE: {val_mse}")
    
    # Store the results
    results.append({
        'lambda': lambda_reg,
        'val_mse': val_mse,
        'alpha': alpha.item(),
        'beta_user': beta_user.detach().clone(),
        'beta_item': beta_item.detach().clone()
    })

# Print the results
print("\nLambda vs Validation MSE:")
for res in results:
    print(f"Lambda: {res['lambda']}, Validation MSE: {res['val_mse']}")


Testing lambda = 0.001
Validation MSE: 1.475295066833496

Testing lambda = 0.01
Validation MSE: 1.4748600721359253

Testing lambda = 0.1
Validation MSE: 1.4707632064819336

Testing lambda = 1
Validation MSE: 1.439719557762146

Testing lambda = 5
Validation MSE: 1.4115872383117676

Testing lambda = 10
Validation MSE: 1.433851957321167

Testing lambda = 20
Validation MSE: 1.4809942245483398

Testing lambda = 50
Validation MSE: 1.5545440912246704

Testing lambda = 100
Validation MSE: 1.6021572351455688

Lambda vs Validation MSE:
Lambda: 0.001, Validation MSE: 1.475295066833496
Lambda: 0.01, Validation MSE: 1.4748600721359253
Lambda: 0.1, Validation MSE: 1.4707632064819336
Lambda: 1, Validation MSE: 1.439719557762146
Lambda: 5, Validation MSE: 1.4115872383117676
Lambda: 10, Validation MSE: 1.433851957321167
Lambda: 20, Validation MSE: 1.4809942245483398
Lambda: 50, Validation MSE: 1.5545440912246704
Lambda: 100, Validation MSE: 1.6021572351455688


In [63]:
lamb = 5
validMSE = 1.4115872383117676

In [64]:
answers['Q8'] = (lamb, validMSE)

In [65]:
assertFloat(answers['Q8'][0])
assertFloat(answers['Q8'][1])

In [68]:
# Set lambda to 5
lambda_reg = 5.0

# Initialize parameters
alpha = torch.nn.Parameter(torch.tensor(0.0))
beta_user = torch.nn.Parameter(torch.zeros(num_users))
beta_item = torch.nn.Parameter(torch.zeros(num_items))

# Set up optimizer
optimizer = torch.optim.LBFGS([alpha, beta_user, beta_item])

# Define objective function with lambda = 5
def closure():
    optimizer.zero_grad()
    predictions = alpha + beta_user[train_users] + beta_item[train_items]
    errors = train_ratings - predictions
    loss = (errors ** 2).sum()
    # Regularization term (excluding alpha)
    reg_term = lambda_reg * ((beta_user ** 2).sum() + (beta_item ** 2).sum())
    total_loss = loss + reg_term
    total_loss.backward()
    return total_loss

# Train the model
optimizer.step(closure)

# After training, detach parameters from computation graph
alpha_value = alpha.detach().item()
beta_user_values = beta_user.detach()
beta_item_values = beta_item.detach()


In [71]:
predictions = open("predictions_Rating.csv", 'w')

# Open the pairs file and read line by line
with open("assignment1/pairs_Rating.csv", 'r') as pairs_file:
    for line in pairs_file:
        if line.startswith("userID"):  # Header
            predictions.write(line)
            continue
        userID, itemID = line.strip().split(',')

        # Handle user index
        if userID in user2index:
            user_idx = user2index[userID]
            beta_u = beta_user_values[user_idx].item()
        else:
            beta_u = 0.0  # Default value for unseen users

        # Handle item index
        if itemID in item2index:
            item_idx = item2index[itemID]
            beta_i = beta_item_values[item_idx].item()
        else:
            beta_i = 0.0  # Default value for unseen items

        # Compute the predicted rating
        pred_rating = alpha_value + beta_u + beta_i

        # Optional: Clip the predicted rating to the valid range (e.g., 1 to 5)
        # pred_rating = max(1.0, min(5.0, pred_rating))

        # Write the prediction
        predictions.write(f"{userID},{itemID},{pred_rating}\n")

# Close the predictions file
predictions.close()

In [67]:
f = open("answers_hw3.txt", 'w')
f.write(str(answers) + '\n')
f.close()